In [31]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import line_search
import random

# Function

In [32]:
# Define the Rosenbrock function
def rosenbrock(x):
    """
    Rosenbrock function.
    f(x, y) = (a - x)^2 + b * (y - x^2)^2
    Commonly, a=1 and b=100.
    """
    a = 1.0
    b = 100.0
    return (a - x[0]) ** 2 + b * (x[1] - x[0] ** 2) ** 2


# Define the gradient of the Rosenbrock function
def rosenbrock_gradient(x):
    """
    Gradient of the Rosenbrock function.
    df/dx = -2*(a-x) - 2*b*(y-x^2)*2*x
    df/dy = 2*b*(y-x^2)
    """
    a = 1.0
    b = 100.0
    df_dx = -2 * (a - x[0]) - 4 * b * x[0] * (x[1] - x[0] ** 2)
    df_dy = 2 * b * (x[1] - x[0] ** 2)
    return np.array([df_dx, df_dy])


# Define the Hessian of the Rosenbrock function
def rosenbrock_hessian(x):
    """
    Hessian of the Rosenbrock function.
    d2f/dx2 = 2 - 4*b*(y-3*x^2)
    d2f/dy2 = 2*b
    d2f/dxdy = -4*b*x
    d2f/dydx = -4*b*x
    """
    a = 1.0
    b = 100.0
    h11 = 2 - 4 * b * (x[1] - 3 * x[0] ** 2)
    h12 = -4 * b * x[0]
    h21 = -4 * b * x[0]
    h22 = 2 * b
    return np.array([[h11, h12], [h21, h22]])


# Golden Section Search for optimal step size
def golden_section_search(func, x_k, p_k, a=0, b=1, tol=1e-5, max_iter=100):
    """
    Golden Section Search for finding the optimal step size.
    Minimizes func(x_k + alpha * p_k) with respect to alpha.

    Args:
        func: The objective function.
        x_k: Current point.
        p_k: Search direction.
        a: Lower bound for alpha.
        b: Upper bound for alpha.
        tol: Tolerance for the search interval.
        max_iter: Maximum number of iterations.

    Returns:
        The optimal step size alpha.
    """
    phi = (1 + np.sqrt(5)) / 2  # Golden ratio
    resphi = 2 - phi

    x1 = a + resphi * (b - a)
    x2 = b - resphi * (b - a)

    f1 = func(x_k + x1 * p_k)
    f2 = func(x_k + x2 * p_k)

    for i in range(max_iter):
        if abs(b - a) < tol:
            return (a + b) / 2

        if f1 < f2:
            b = x2
            x2 = x1
            f2 = f1
            x1 = a + resphi * (b - a)
            f1 = func(x_k + x1 * p_k)
        else:
            a = x1
            x1 = x2
            f1 = f2
            x2 = b - resphi * (b - a)
            f2 = func(x_k + x2 * p_k)

    return (a + b) / 2



In [33]:

random_state = 42
np.random.seed(random_state)

initial_x = np.random.uniform(-2, 2, 2)
print(f"Initial approximation (random_state={random_state}): {initial_x}")
print(f"Initial function value: {rosenbrock(initial_x)}")

FUNCTION_TOLERANCE = 10e-5

Initial approximation (random_state=42): [-0.50183952  1.80285723]
Initial function value: 242.82006316689603


In [34]:

# --- Newton's Method Definition ---
def newton_method(
    func, grad, hess, x0, tol=FUNCTION_TOLERANCE, max_iter=1000, line_search_method=None
):
    """
    Newton's Method for unconstrained optimization.

    Args:
        func: The objective function.
        grad: The gradient of the objective function.
        hess: The Hessian of the objective function.
        x0: Initial guess.
        tol: Tolerance for the function value.
        max_iter: Maximum number of iterations.
        line_search_method: Function for line search (e.g., golden_section_search).

    Returns:
        x_k: The found minimum point.
        f_values: List of function values at each iteration.
        iterations: Number of iterations performed.
        operations: Dictionary counting function, gradient, Hessian evaluations, and inverse operations.
    """
    x_k = np.array(x0, dtype=float)
    f_values = [func(x_k)]
    operations = {"func_evals": 1, "grad_evals": 0, "hess_evals": 0, "inv_ops": 0}

    for i in range(max_iter):
        gradient = grad(x_k)
        hessian = hess(x_k)
        operations["grad_evals"] += 1
        operations["hess_evals"] += 1

        try:
            hessian_inv = np.linalg.inv(hessian)
            operations["inv_ops"] += 1
        except np.linalg.LinAlgError:
            # print(f"Newton's Method: Hessian is singular at iteration {i}. Stopping.") # Removed print
            break

        p_k = -np.dot(hessian_inv, gradient)

        alpha = 1.0

        if line_search_method:
            alpha = line_search_method(func, x_k, p_k)
            if alpha is None:
                # print(f"Newton's Method: Line search failed at iteration {i}. Stopping.") # Removed print
                break

        x_k = x_k + alpha * p_k
        current_f_value = func(x_k)
        f_values.append(current_f_value)
        operations["func_evals"] += 1

        if current_f_value < tol:
            # print(f"Newton's Method converged to f(x) < {tol} at iteration {i+1}.") # Removed print
            break
    else:
        pass  # print(f"Newton's Method reached max iterations ({max_iter}).") # Removed print

    return x_k, f_values, i + 1, operations


# --- Newton's Method Usage ---
print("\n--- Running Newton's Method ---")
x_newton, f_values_newton, iter_newton, ops_newton = newton_method(
    rosenbrock,
    rosenbrock_gradient,
    rosenbrock_hessian,
    initial_x,
    line_search_method=golden_section_search,
)
print(
    f"Newton's Method Result: x={x_newton}, f(x)={f_values_newton[-1]:.6f}, Iterations: {iter_newton}, Operations: {ops_newton}"
)



--- Running Newton's Method ---
Newton's Method Result: x=[0.99558475 0.99108911], f(x)=0.000020, Iterations: 15, Operations: {'func_evals': 16, 'grad_evals': 15, 'hess_evals': 15, 'inv_ops': 15}


In [35]:
# --- Nesterov's Accelerated Gradient Method Definition ---
def nesterov_method(
    func, grad, x0, tol=FUNCTION_TOLERANCE, max_iter=1000, learning_rate=0.01
):
    """
    Nesterov's Accelerated Gradient (NAG) Method.

    Args:
        func: The objective function.
        grad: The gradient of the objective function.
        x0: Initial guess.
        tol: Tolerance for the function value.
        max_iter: Maximum number of iterations.
        learning_rate: Initial learning rate (alpha).

    Returns:
        x_k: The found minimum point.
        f_values: List of function values at each iteration.
        iterations: Number of iterations performed.
        operations: Dictionary counting function and gradient evaluations.
    """
    x_k = np.array(x0, dtype=float)
    v_k = np.zeros_like(x0, dtype=float)  # Momentum term
    f_values = [func(x_k)]
    operations = {"func_evals": 1, "grad_evals": 0}

    for i in range(max_iter):
        t_k = (i + 1) / 2  # Simple Nesterov's coefficient
        y_k = x_k + (t_k / (t_k + 3)) * v_k  # Nesterov's acceleration

        gradient = grad(y_k)
        operations["grad_evals"] += 1

        # Check for NaN or inf in gradient
        if np.isnan(gradient).any() or np.isinf(gradient).any():
            print(
                f"Nesterov's Method: NaN or Inf encountered in gradient at iteration {i}. Stopping."
            )
            break

        v_k = v_k - learning_rate * gradient

        # Check for NaN or inf in v_k
        if np.isnan(v_k).any() or np.isinf(v_k).any():
            print(
                f"Nesterov's Method: NaN or Inf encountered in momentum at iteration {i}. Stopping."
            )
            break

        x_k = x_k + v_k

        current_f_value = func(x_k)
        f_values.append(current_f_value)
        operations["func_evals"] += 1

        # Check for NaN or inf in function value
        if np.isnan(current_f_value) or np.isinf(current_f_value):
            print(
                f"Nesterov's Method: NaN or Inf encountered in function value at iteration {i}. Stopping."
            )
            break

        if current_f_value < tol:
            # print(f"Nesterov's Method converged to f(x) < {tol} at iteration {i+1}.") # Removed print
            break
    else:
        pass  # print(f"Nesterov's Method reached max iterations ({max_iter}).") # Removed print

    return x_k, f_values, i + 1, operations


# --- Nesterov's Method Usage ---
print("\n--- Running Nesterov's Method ---")
# Adjusted learning rate for Rosenbrock to prevent overflow/NaN
x_nesterov, f_values_nesterov, iter_nesterov, ops_nesterov = nesterov_method(
    rosenbrock,
    rosenbrock_gradient,
    initial_x,
    learning_rate=0.00005,  # Further reduced learning rate
)
print(
    f"Nesterov's Method Result: x={x_nesterov}, f(x)={f_values_nesterov[-1]:.6f}, Iterations: {iter_nesterov}, Operations: {ops_nesterov}"
)



--- Running Nesterov's Method ---
Nesterov's Method Result: x=[-3.72865633 13.90910115], f(x)=22.364063, Iterations: 1000, Operations: {'func_evals': 1001, 'grad_evals': 1000}


In [36]:
# --- SR1 (Symmetric Rank-1) Quasi-Newton Method Definition ---
def sr1_method(
    func, grad, x0, tol=FUNCTION_TOLERANCE, max_iter=1000, line_search_method=None
):
    """
    Symmetric Rank-1 (SR1) Quasi-Newton Method.
    Approximates the inverse Hessian.

    Args:
        func: The objective function.
        grad: The gradient of the objective function.
        x0: Initial guess.
        tol: Tolerance for the function value.
        max_iter: Maximum number of iterations.
        line_search_method: Function for line search (e.g., golden_section_search).

    Returns:
        x_k: The found minimum point.
        f_values: List of function values at each iteration.
        iterations: Number of iterations performed.
        operations: Dictionary counting function and gradient evaluations.
    """
    x_k = np.array(x0, dtype=float)
    n = len(x0)
    H_k = np.eye(n)  # Initial approximation of the inverse Hessian

    f_values = [func(x_k)]
    operations = {
        "func_evals": 1,
        "grad_evals": 0,
        "inv_ops": 0,
    }  # No hess_evals for quasi-Newton

    prev_grad = grad(x_k)
    operations["grad_evals"] += 1

    for i in range(max_iter):
        p_k = -np.dot(H_k, prev_grad)

        alpha = 1.0

        if line_search_method:
            alpha = line_search_method(func, x_k, p_k)
            if alpha is None:
                # print(f"SR1 Method: Line search failed at iteration {i}. Stopping.") # Removed print
                break

        s_k = alpha * p_k  # Step taken
        x_k_plus_1 = x_k + s_k

        current_grad = grad(x_k_plus_1)
        operations["grad_evals"] += 1

        y_k = current_grad - prev_grad  # Change in gradient

        Hy = np.dot(H_k, y_k)
        denominator = np.dot(y_k, s_k) - np.dot(y_k, Hy)

        if (
            abs(denominator) > 1e-8
        ):  # A small threshold to prevent numerical instability
            term = s_k - Hy
            H_k = H_k + np.outer(term, term) / denominator
        else:
            # print(f"SR1 Method: Skipping update due to small denominator at iteration {i}.") # Removed print
            pass

        x_k = x_k_plus_1
        prev_grad = current_grad

        current_f_value = func(x_k)
        f_values.append(current_f_value)
        operations["func_evals"] += 1

        if current_f_value < tol:
            # print(f"SR1 Method converged to f(x) < {tol} at iteration {i+1}.") # Removed print
            break
    else:
        pass  # print(f"SR1 Method reached max iterations ({max_iter}).") # Removed print

    return x_k, f_values, i + 1, operations

# --- SR1 Method Usage ---
print("\n--- Running SR1 Method ---")
x_sr1, f_values_sr1, iter_sr1, ops_sr1 = sr1_method(
    rosenbrock, rosenbrock_gradient, initial_x, line_search_method=golden_section_search
)
print(
    f"SR1 Method Result: x={x_sr1}, f(x)={f_values_sr1[-1]:.6f}, Iterations: {iter_sr1}, Operations: {ops_sr1}"
)



--- Running SR1 Method ---
SR1 Method Result: x=[0.11039602 0.0029492 ], f(x)=0.799929, Iterations: 1000, Operations: {'func_evals': 1001, 'grad_evals': 1001, 'inv_ops': 0}
